# 分类算法

当训练数据中某些类型的频率明显高于其他类型时,仅仅使用准确率(accuracy)来衡量模型效果会造成很大的偏差

## 混淆矩阵(Confusion Matrix)

以随机梯度下降法(Stochastic Gradient Descent)为例:

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train)

In [ ]:
#交叉预测
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train, cv=3)

与cross_val_score()类似,cross_val_predict()使用K-fold交叉验证  
但cross_val_predict()返回的是每个test fold的预测值,而非评估分值

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, y_train_pred)

在混淆矩阵中,  
每一行表示一个真实分类,每一列表示预测的分类

矩阵的第一行表示实际分类为-1(negative class),其中2000个为正确分类为-1(true negative),200个被错误分类为1(false positive)  
矩阵的第二行表示实际分类为1(positive class),其中250个被错误分类为-1(false negative),800个被正确分类为1(true positive)

$\large precision=\frac{TP}{TP+FP}$  
$\large recall=\frac{TP}{TP+FN}$  ,也称为sensitivity, true positive rate(TPR)

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train, y_train_pred    #计算precision
recall_score(y_train, y_train_pred)      #计算recall

为了便于衡量2个分类器,通常可以将precision和recall转化成一个标准,称为F1  
F1为precision和recall的调和平均值(harmonic mean),调和平均值为较低的值赋予更高的权重,只有当2个分值都较高时,F1才会较高
F1更倾向于给具有相似分值的precision和recall一个相对更高的分值, 但这与实际建模的目的可能存在一定的不符

$\Large F1=\frac{2}{\frac{1}{precision}+\frac{1}{recall}}=2×\frac{precision×recall}{precision+recall}$

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train, y_train_pred)   # 计算F1值

precision和recall是一个此消彼长的关系,通过直接设置threshold,可以选取任意的决策分值来使模型进行分类

In [ ]:
y_scores = sgd_clf.decision_function([some_digit])   #decision_function返回实例的分值
y_scores     

In [ ]:
threshold = 0
y_some_digit_pred = (y_scores > threshold)

### Precision and recall versus the decision threshold

为了决定究竟应该使用多大的threshold,需要获取所有实例的分值

In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train, cv=3,
                             method="decision_function")   #返回分值,而非预测的分类结果

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train, y_scores)

使用precision_recall_curve计算所有可能threshold下的precision和recall

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1])

plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.xlim([-700000, 700000])
save_fig("precision_recall_vs_threshold_plot")
plt.show()

绘制所有可能threshold下的precision和recall图

### PR(Precision versus recall)曲线

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.axis([0, 1, 0, 1])

plt.figure(figsize=(8, 6))
plot_precision_vs_recall(precisions, recalls)
save_fig("precision_vs_recall_plot")

在有些情况下,可以通过绘制precision versus recall图,选择在precision将要大幅度下降时的recall值

### THE ROC(receiver operating characterisitc) Curve

ROC绘制的是以False Positive Rate为x轴, True Positive Rate为y轴的曲线  
其中,  
$True Positive Rate(recall)=\frac{TP}{TP+FN}$  
$False Positive Rate=\frac{FP}{TN+FP}$

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train, y_scores)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)
save_fig("roc_curve_plot")
plt.show()

在ROC中又有一个权衡关系,recall(TPR)越高,FPR也越高  
一个好的ROC曲线,应该是趋近于左上角,ROC曲线下的面积称为AUC(area under the curve)

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_train, y_scores)   #计算ROC曲线的AUC面积

完美分类器的AUC面积为1, 分类器AUC越靠近1,效果越好  
当positive值在样本中的比例较小,或者更关心FP而非FN时,应该优先选用PR(precision vs recall)曲线,而不是ROC曲线  
注:ROC和ROC AUC分值可以用来比较不同模型之间的分类效果

## 多类别分类器(Multiclass Classification)

一些算法,如随机森林,朴素贝叶斯分类器支持多类别分类  
而另一些算法,如SVM,线性分类器则只支持二元分类,但通过使用多个二元分类,这些算法也能进行多类别分类

### one-versus-all(OvA)策略

根据需要分类的类别数量n,创建n个分类器  
哪一个分类器的分值最高,则最终结果属于该类别  
对于大多数算法来说,推荐使用OvA

### one-versus-one(OvO)策略

对每一个类别对进行一一分类  
以区分0~2为例:  
训练分类器:0vs1,0vs2,1vs2
如果有n个类别,则需要训练n×(n-1)/2个模型  
对于SVM来说,训练小样本但多个分类器的效率更高,因此推荐使用OvO

以SGD为例:

In [ ]:
sgd_clf.fit(X_train, y_train)  #当标签类别为多元时,建立模型的时候,自动采用OvA策略
sgd_clf.predict([some_digit]) #返回某一个数字的预测值(所有分类器中分值最高的一个)

In [ ]:
some_digit_scores = sgd_clf.decision_function([some_digit])
some_digit_scores

该算法使用OvA策略,decision_function返回所有分类器的分值

In [ ]:
sgd_clf.classes_   #保存了所有分类的值,并按顺序进行排列

### 强制使用某一种策略

使用OneVsOneClassifier或OneVsRestClassifier

In [ ]:
from sklearn.multiclass import OneVsOneClassifier

ovo_clf = OneVsOneClassifier(SGDClassifier(random_state=42))
ovo_clf.fit(X_train, y_train)
ovo_clf.predict([some_digit])